In [1]:
%load_ext autoreload
%autoreload 2

<h3> Загрузка библиотек

In [2]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR

In [3]:
from ml.models.unet3d import U_Net
from ml.models.rog import ROG
from ml.models.unet_deepsup import Unet_MSS
from ml.models.LiquidNet import LiquidNet, LiquidNetBlock, conv_block, bottle_neck_connection
from ml.models.LiquidNet_configs.second_config import get_config

from ml.models.activates import swish
from ml.utils import get_total_params, load_pretrainned
from ml.tio_dataset import TioDataset
from ml.controller import Controller
from ml.losses import (ExponentialLogarithmicLoss, WeightedExpBCE, TverskyLoss,
                       IOU_Metric, MultyscaleLoss, SumLoss, LinearCombLoss)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
train_settings  = {
    "patch_shape" : (64, 64, 64),
    "patches_per_volume" : 128,
    "patches_queue_length" : 1440,
    "batch_size" : 8,
    "num_workers": 4,
    "sampler": "weighted" #"uniform",#
}

val_settings  = {
    "patch_shape" : (64, 64, 64),
    "patches_per_volume" : 32,
    "patches_queue_length" : 1440,
    "batch_size" : 8,
    "num_workers": 4,
    "sampler": "uniform",#"weighted" #"uniform",#
}

test_settings  = {
    "patch_shape" : (256, 256, 128),
    "overlap_shape" : (32, 32, 24),
    "batch_size" : 1,
    "num_workers": 4,
}

data_dir = "/home/msst/Documents/medtech/MainData"
dataset = TioDataset(data_dir,
                 train_settings=train_settings,
                 val_settings=val_settings,
                 test_settings=test_settings)

In [6]:
# test_settings  = {
#     "patch_shape" : (128, 128, 128),
#     "overlap_shape" : (32, 32, 24),
#     "batch_size" : 1,
#     "num_workers": 4,
# }

# data_dir = "/home/msst/Documents/medtech/MainData_test"
# dataset = TioDataset(data_dir,
#                      train_settings=None,
#                      val_settings=None,
#                      test_settings=test_settings)

In [7]:
# model = Unet_MSS(channel_coef=32, act_fn=swish())

In [10]:
model = LiquidNet(*get_config(16), debug=False)
print("total_params:", get_total_params(model))
#model.net_graph

total_params: 5304803


In [11]:
# funcs_and_сoef_list = []

# funcs_and_сoef_list.append([ExponentialLogarithmicLoss(gamma_tversky = 1, gamma_bce = 1, lamb=0.0,
#                                    freq = 0.001, tversky_alfa=0.75), 1])

# funcs_and_сoef_list.append([TverskyLoss(0.55), 1])


# funcs_and_сoef_list.append([SumLoss(alfa=0.5), 0.1])

# loss_fn = MultyscaleLoss(LinearCombLoss(funcs_and_сoef_list))

In [12]:
loss_fn = MultyscaleLoss(TverskyLoss(0.55))

In [15]:
#loss_fn = MultyscaleLoss(ExponentialLogarithmicLoss(gamma_tversky = 2, gamma_bce = 2, lamb=0.7,
#                                                    freq = 0.001, tversky_alfa=0.5))


metric_fn = IOU_Metric()

controller_config = {
    "loss" : loss_fn,
    "metric" : metric_fn,
    'device' : device,
    "optimizer_fn" : lambda model: torch.optim.ASGD(model.parameters(), lr=0.001),
    "sheduler_fn": lambda optimizer: StepLR(optimizer, step_size=5, gamma=0.5)
}
controller = Controller(controller_config)

cuda


In [ ]:
controller.fit(model, dataset, 50)

Epoch 1/50


 92%|███████████████████████████████████████▋   | 59/64 [00:39<00:03,  1.51it/s]Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.8/

{'mean_loss': 1.434290248900652}


100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.15it/s]


{'mean_loss': 1.4454477578401566}


...

100%|███████████████████████████████████████████| 64/64 [01:01<00:00,  1.04it/s]


{'mean_loss': 1.4331202805042267}


100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.13it/s]


{'mean_loss': 1.44960318505764}


  0%|                                                     | 0/2 [00:00<?, ?it/s]

In [32]:
#model_name = "LiquidNet_second_config_8_ExpLog09_Tversky055_epoch50"
#controller.save("/home/msst/repo/MSRepo/VesselSegmentation/saved_models/" + model_name)

In [30]:
#controller.load_model(model, "/home/msst/repo/MSRepo/VesselSegmentation/saved_models/" + model_name)
#controller.model = model.to(device)

In [33]:
path_to_check= "/home/msst/repo/MSRepo/VesselSegmentation/saved_models/" + model_name
controller.load(model, path_to_checkpoint=path_to_check)

In [12]:
controller.val_epoch(dataset.test_dataloader)

100%|█████████████████████████████████████████████| 7/7 [01:15<00:00, 10.74s/it]


{'metrics': [{'sample': 'P62_CTA',
   'seg_sum/GT_sum': tensor(0.0060),
   'metric1': tensor([7.3981e-11])},
  {'sample': 'P28_CTA',
   'seg_sum/GT_sum': tensor(0.0002),
   'metric1': tensor([0.0011])},
  {'sample': 'P12_CTA',
   'seg_sum/GT_sum': tensor(74.4396),
   'metric1': tensor([0.0129])},
  {'sample': 'P70_CTA',
   'seg_sum/GT_sum': tensor(0.0044),
   'metric1': tensor([0.0011])},
  {'sample': 'P35_CTA',
   'seg_sum/GT_sum': tensor(0.0040),
   'metric1': tensor([0.0006])},
  {'sample': 'new_CTA',
   'seg_sum/GT_sum': tensor(0.0035),
   'metric1': tensor([7.4289e-05])},
  {'sample': 'CT_S5020',
   'seg_sum/GT_sum': tensor(36.7029),
   'metric1': tensor([0.0207])}]}

In [24]:
model_name = 'test'
data_dir = "seg_data/" + model_name
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
controller.predict(dataset.test_dataloader, data_dir)

100%|█████████████████████████████████████████████| 2/2 [00:21<00:00, 10.97s/it]


[{'sample': 'P62_CTA_0',
  'seg_sum/GT_sum': tensor(0.5912),
  'metric1': tensor([0.0703])},
 {'sample': 'new_CTA_0',
  'seg_sum/GT_sum': tensor(0.5257),
  'metric1': tensor([0.0477])}]